In [ ]:
import torch
import openvino as ov
useOpenVino=False
if (torch.cuda.is_available()):
    torch.set_default_device('cuda:0')
else: 
    print('OpenVINO Devices: ' ,ov.Core().available_devices)
    ovDevice=ov.Core().available_devices[-1]
    print("Using OpenVINO device "+ovDevice)
    useOpenVino=True
print('CUDA available: '+str(torch.cuda.is_available()))
print("Using OpenVino: "+ str(useOpenVino))

# model_name='./data/qwen_fixed';
model_name='./data/fine_tuned_qwen_adapter2';

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
if not useOpenVino:
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from optimum.intel import OVModelForCausalLM, OVWeightQuantizationConfig
from transformers import AutoTokenizer

if useOpenVino:
    quantization_config = OVWeightQuantizationConfig(bits=4)
    model = OVModelForCausalLM.from_pretrained(model_name, library='transformers',device=ovDevice)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model = AutoModelForCausalLM.from_pretrained('./data/qwen_fixed')
# tokenizer = AutoTokenizer.from_pretrained('./data/qwen_fixed')

In [ ]:
from datasets import Dataset
ds=Dataset.from_json('data/data.json')

In [ ]:
print(ds)

In [ ]:
import torch
import time
fim_prefix_id = tokenizer.convert_tokens_to_ids("<|fim_prefix|>")
fim_suffix_id = tokenizer.convert_tokens_to_ids("<|fim_suffix|>")
fim_middle_id = tokenizer.convert_tokens_to_ids("<|fim_middle|>")
fim_pad_id = tokenizer.convert_tokens_to_ids("<|fim_pad|>")
endoftext_id = tokenizer.convert_tokens_to_ids("<|endoftext|>")
plus_ids = tokenizer("+++")["input_ids"]
newline_ids = tokenizer("\n")["input_ids"]
space_ids = tokenizer(" ")["input_ids"]

def padToLength(list,length, padding):
    result=list[:length]
    return [padding]*(length-len(result))+result;

def generate_response(examples):
    # Tokenize all prefixes and suffixes together
    language_ids = tokenizer(examples["language"], add_special_tokens=False, split_special_tokens=True)["input_ids"]
    path_ids = tokenizer(['/'.join(path.split('/')[-5:]) for path in examples["fileName"]], add_special_tokens=False, split_special_tokens=True)["input_ids"]
    prefix_ids = tokenizer(examples["prefix"], add_special_tokens=False, split_special_tokens=True)["input_ids"]
    suffix_ids = tokenizer(examples["suffix"], add_special_tokens=False,split_special_tokens=True)["input_ids"]


    # Combine the IDs for each example in the batch
    prompt_ids = [
       [fim_prefix_id]+plus_ids+language+space_ids+path+ newline_ids+ prefix + [fim_suffix_id] + suffix +[fim_middle_id]
       for language, path,prefix, suffix in zip(language_ids, path_ids,prefix_ids, suffix_ids)
    ]

    attention_mask = [[1] * len(ids) for ids in prompt_ids]

    # pad batch
    max_length= max_length = max([len(ids) for ids in prompt_ids])
    p_prompt_ids=[padToLength(ids,max_length, fim_pad_id) for ids in prompt_ids]
    p_attention_mask=[padToLength(ids,max_length, 0) for ids in attention_mask]

    outputs = model.generate(**{
        "input_ids": torch.tensor(p_prompt_ids, dtype=torch.int64),
        "attention_mask":torch.tensor(p_attention_mask, dtype=torch.int64)
    },max_new_tokens=20,eos_token_id=[endoftext_id,fim_pad_id ] ,pad_token_id=fim_pad_id)
    outputs = [out[max_length:] for out,prompt in zip(outputs,prompt_ids)]
    response= [tokenizer.decode(ids, clean_up_tokenization_spaces=True, skip_special_tokens=True) for ids in outputs]
    return {
        'response': response, 
        'match': [r.startswith(exp) for r,exp in zip(response, examples['expected'])]
    }

ds2=ds.shuffle(seed=42).map(generate_response, batched=True, batch_size=8)

In [9]:
import json
rows=[]
for example in ds2:
    # print('PREFIX: '+example['prefix'])
    # print('RESPOSE: '+example['response'])
    # print('MATCH: '+str(example['match']))
    row=example['parameters'].copy()
    row['match']=example['match']
    rows.append(row)
# print(rows)
with open('data/results1.json', 'w') as file:
    json.dump(rows, file, indent=2)

In [9]:
# ds2.to_json('data/evaluation.json')